# 1er assaie

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape


# Charger le jeu de données MNIST
(x_train, _), (_, _) = mnist.load_data()

# Normaliser les pixels entre 0 et 1
x_train = x_train.astype('float32') / 255.0

# Diviser le jeu de données en ensembles d'entraînement et de test
X_train, X_test, _, _ = train_test_split(x_train, x_train, test_size=0.2, random_state=42)

# Aplatir les images et les normaliser
X_train = X_train.reshape((len(X_train), 28, 28, 1))
X_test = X_test.reshape((len(X_test), 28, 28, 1))
X_train = X_train / 255.0
X_test = X_test / 255.0

# Créer un autoencodeur convolutionnel
input_img = Input(shape=(28, 28, 1))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Entraîner l'autoencodeur sur le jeu d'entraînement (sans anomalies)
autoencoder.fit(X_train, X_train, epochs=10, batch_size=128, shuffle=True, validation_data=(X_test, X_test))

# Encoder de l'autoencodeur (utilisé comme détecteur d'anomalies)
encoder = Model(input_img, encoded)

# Obtenir les représentations encodées des images d'entraînement
X_train_encoded = encoder.predict(X_train)

# Aplatir les représentations encodées
X_train_encoded_flat = X_train_encoded.reshape((len(X_train_encoded), np.prod(X_train_encoded.shape[1:])))

# Entraîner le modèle One-Class SVM sur les représentations encodées
clf = OneClassSVM(nu=0.1)
clf.fit(X_train_encoded_flat)

# Prédiction sur le jeu de test
X_test_encoded = encoder.predict(X_test)
X_test_encoded_flat = X_test_encoded.reshape((len(X_test_encoded), np.prod(X_test_encoded.shape[1:])))
y_pred = clf.predict(X_test_encoded_flat)

# Étiquettes réelles du jeu de test
y_true = np.ones_like(y_pred)

# Évaluer la précision sur le jeu de test
accuracy_test = accuracy_score(y_true, y_pred)
print(f"Accuracy on test set: {accuracy_test}")


2023-11-24 11:31:58.281298: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-24 11:31:58.284403: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-24 11:31:58.337764: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 11:31:58.337807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 11:31:58.338919: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/10
375/375 [==============================] - 11s 27ms/step - loss: 0.0493 - val_loss: 0.0042
Epoch 2/10
375/375 [==============================] - 10s 26ms/step - loss: 0.0040 - val_loss: 0.0040
Epoch 3/10
375/375 [==============================] - 10s 26ms/step - loss: 0.0040 - val_loss: 0.0040
Epoch 4/10
375/375 [==============================] - 10s 25ms/step - loss: 0.0040 - val_loss: 0.0040
Epoch 5/10
375/375 [==============================] - 10s 26ms/step - loss: 0.0040 - val_loss: 0.0040
Epoch 6/10
375/375 [==============================] - 10s 25ms/step - loss: 0.0040 - val_loss: 0.0040
Epoch 7/10
375/375 

In [6]:
pip install tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 109.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 52.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 118.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 94.5 MB/s eta 0:00:00
     ━━

# 2eme essaie

### Normaliser le jeu de données MNIST

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape

# Charger le jeu de données MNIST
(x_train, _), (_, _) = mnist.load_data()

x_train.shape

# Normaliser les pixels entre 0 et 1
x_train = x_train.astype('float32') / 255.0

# Diviser le jeu de données en ensembles d'entraînement et de test
x_train, X_test, _, _ = train_test_split(x_train, x_train, test_size=10000, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(x_train, x_train, test_size=12000, random_state=42)
X_train.shape, X_test.shape, X_val.shape

((38000, 28, 28), (10000, 28, 28), (12000, 28, 28))

### générer les images aléatoire

In [20]:

# Ici, j'utilise des images aléatoires, mais vous pouvez les remplacer par votre propre dataset
random_images = np.random.rand(10000, 28, 28)
# anomalies = random_images.reshape((len(random_images), np.prod(random_images.shape[1:])))

anomalies = random_images.astype('float32') / 255.0
anomalies.shape

(10000, 28, 28)

In [21]:

# Concaténer le jeu de données MNIST avec le jeu d'anomalies
x_combined = np.concatenate([X_test, anomalies])
x_combined.shape

# Aplatir les images et les normaliser
X_train = X_train.reshape((len(X_train), 28, 28, 1))
X_val = X_val.reshape((len(X_val), 28, 28, 1))
X_test = x_combined.reshape((len(x_combined), 28, 28, 1))
X_train = X_train / 255.0
X_test = X_test / 255.0
X_val = X_val / 255.0

X_train.shape, X_test.shape, X_val.shape

((38000, 28, 28, 1), (20000, 28, 28, 1), (12000, 28, 28, 1))

In [22]:

# Créer un autoencodeur convolutionnel
input_img = Input(shape=(28, 28, 1))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.fit(X_train, X_train, epochs=10, batch_size=128, shuffle=True, validation_data=(X_val, X_val))

# Encoder de l'autoencodeur (utilisé comme détecteur d'anomalies)
encoder = Model(input_img, encoded)

# Obtenir les représentations encodées des images d'entraînement
X_train_encoded = encoder.predict(X_train)

# Aplatir les représentations encodées
X_train_encoded_flat = X_train_encoded.reshape((len(X_train_encoded), np.prod(X_train_encoded.shape[1:])))

# Entraîner le modèle One-Class SVM sur les représentations encodées
clf = OneClassSVM(nu=0.1)
clf.fit(X_train_encoded_flat)

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape

# Charger le jeu de données MNIST
(x_train, _), (_, _) = mnist.load_data()

# Ici, j'utilise des images aléatoires, mais vous pouvez les remplacer par votre propre dataset
random_images = np.random.rand(10000, 28, 28)
anomalies = random_images.astype('float32') / 255.0
anomalies.shape   # Aplatir les images et les normaliser
X_train = x_train.reshape((len(x_train), 28, 28, 1))
anomalies = anomalies.reshape((len(anomalies), 28, 28, 1))
X_train = X_train / 255.0
anomalies = anomalies / 255.0

# Diviser le jeu de données en ensembles d'entraînement (38 000 images), de validation (12 000 images) et de test (1000 MNIST + 10 000 générées)
X_train, X_val = train_test_split(X_train, test_size=12000, random_state=42)
X_train, X_test = train_test_split(X_train, test_size=10000, random_state=42)
# X_test_anomalies, _ = train_test_split(anomalies, test_size=10000, random_state=42)
X_test_combined = np.concatenate([X_test, anomalies])
X_test_combined.shape, X_train.shape, X_val.shape

((20000, 28, 28, 1), (38000, 28, 28, 1), (12000, 28, 28, 1))

In [ ]:




# Créer un autoencodeur convolutionnel
input_img = Input(shape=(28, 28, 1))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(X_train, X_train, epochs=10, batch_size=128, shuffle=True, validation_data=(X_val, X_val))

# Encoder de l'autoencodeur (utilisé comme détecteur d'anomalies)
encoder = Model(input_img, encoded)

# Obtenir les représentations encodées des images d'entraînement
X_train_encoded = encoder.predict(X_train)

# Aplatir les représentations encodées
X_train_encoded_flat = X_train_encoded.reshape((len(X_train_encoded), np.prod(X_train_encoded.shape[1:])))

# Entraîner le modèle One-Class SVM sur les représentations encodées
clf = OneClassSVM(nu=0.1)
clf.fit(X_train_encoded_flat)

# Prédiction sur l'ensemble de test (MNIST + anomalies)
X_test_mnist_encoded = encoder.predict(X_test_mnist)
X_test_mnist_encoded_flat = X_test_mnist_encoded.reshape((len(X_test_mnist_encoded), np.prod(X_test_mnist_encoded.shape[1:])))
y_pred_mnist = clf.predict(X_test_mnist_encoded_flat)

X_test_anomalies_encoded = encoder.predict(X_test_anomalies)
X_test_anomalies_encoded_flat = X_test_anomalies_encoded.reshape((len(X_test_anomalies_encoded), np.prod(X_test_anomalies_encoded.shape[1:])))
y_pred_anomalies = clf.predict(X_test_anomalies_encoded_flat)

# Étiquettes réelles de l'ensemble de test MNIST
y_true_mnist = np.ones_like(y_pred_mnist)

# Étiquettes réelles de l'ensemble de test anomalies
y_true_anomalies = -1 * np.ones_like(y_pred_anomalies)

# Concaténer les prédictions et les étiquettes réelles
y_pred = np.concatenate([y_pred_mnist, y_pred_anomalies])
y_true = np.concatenate([y_true_mnist, y_true_anomalies])

# Évaluer la précision sur l'ensemble de test
accuracy_test = accuracy_score(y_true, y_pred)
print(f"Accuracy on test set: {accuracy_test}")



In [27]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape


# Charger le jeu de données MNIST
(x_train, _), (_, _) = mnist.load_data()

# Ici, j'utilise des images aléatoires, mais vous pouvez les remplacer par votre propre dataset
random_images = np.random.rand(10000, 28, 28)
anomalies = random_images.astype('float32') / 255.0
anomalies.shape   # Aplatir les images et les normaliser
X_train = x_train.reshape((len(x_train), 28, 28, 1))
anomalies = anomalies.reshape((len(anomalies), 28, 28, 1))
X_train = X_train / 255.0
anomalies = anomalies / 255.0

# Diviser le jeu de données en ensembles d'entraînement (38 000 images), de validation (12 000 images) et de test (1000 MNIST + 10 000 générées)
X_train, X_temp = train_test_split(X_train, test_size=12000, random_state=42)
X_train, X_test = train_test_split(X_temp, test_size=10000, random_state=42)

# Concaténer les images de test MNIST avec les anomalies générées
X_test_combined = np.concatenate([X_test, anomalies])

X_train.shape, X_test_combined.shape

((2000, 28, 28, 1), (20000, 28, 28, 1))

In [30]:
X_train, X_temp = train_test_split(X_train, test_size=12000, random_state=42)
X_val, X_test = train_test_split(X_temp, test_size=10000, random_state=42)
X_train.shape,X_val.shape, X_test.shape

ValueError: test_size=12000 should be either positive and smaller than the number of samples 2000 or a float in the (0, 1) range

In [28]:
# Créer un autoencodeur convolutionnel
input_img = Input(shape=(28, 28, 1))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(X_train, X_train, epochs=10, batch_size=128, shuffle=True, validation_split=0.1)

Epoch 1/10
15/15 [==============================] - 1s 36ms/step - loss: 0.5951 - val_loss: 0.4776
Epoch 2/10
15/15 [==============================] - 0s 26ms/step - loss: 0.4528 - val_loss: 0.4053
Epoch 3/10
15/15 [==============================] - 0s 26ms/step - loss: 0.3664 - val_loss: 0.3188
Epoch 4/10
15/15 [==============================] - 0s 26ms/step - loss: 0.2834 - val_loss: 0.2553
Epoch 5/10
15/15 [==============================] - 0s 27ms/step - loss: 0.2400 - val_loss: 0.2333
Epoch 6/10
15/15 [==============================] - 0s 26ms/step - loss: 0.2253 - val_loss: 0.2249
Epoch 7/10
15/15 [==============================] - 0s 26ms/step - loss: 0.2167 - val_loss: 0.2159
Epoch 8/10
15/15 [==============

In [29]:
X_train.shape

(2000, 28, 28, 1)

In [26]:




# Encoder de l'autoencodeur (utilisé comme détecteur d'anomalies)
encoder = Model(input_img, encoded)

# Obtenir les représentations encodées des images d'entraînement
X_train_encoded = encoder.predict(X_train)

# Aplatir les représentations encodées
X_train_encoded_flat = X_train_encoded.reshape((len(X_train_encoded), np.prod(X_train_encoded.shape[1:])))

# Entraîner le modèle One-Class SVM sur les représentations encodées
clf = OneClassSVM(nu=0.1)
clf.fit(X_train_encoded_flat)

# Prédiction sur l'ensemble de test (MNIST + anomalies)
X_test_combined_encoded = encoder.predict(X_test_combined)
X_test_combined_encoded_flat = X_test_combined_encoded.reshape((len(X_test_combined_encoded), np.prod(X_test_combined_encoded.shape[1:])))
y_pred_combined = clf.predict(X_test_combined_encoded_flat)

# Étiquettes réelles de l'ensemble de test (MNIST + anomalies)
y_true_combined = np.concatenate([np.ones(len(X_test)), -1 * np.ones(len(anomalies))])

# Évaluer la précision sur l'ensemble de test
accuracy_test = accuracy_score(y_true_combined, y_pred_combined)
print(f"Accuracy on combined test set: {accuracy_test}")


Epoch 1/10
15/15 [==============================] - 1s 39ms/step - loss: 0.6759 - val_loss: 0.6128
Epoch 2/10
15/15 [==============================] - 0s 25ms/step - loss: 0.5114 - val_loss: 0.4491
Epoch 3/10
15/15 [==============================] - 0s 26ms/step - loss: 0.4096 - val_loss: 0.3625
Epoch 4/10
15/15 [==============================] - 0s 25ms/step - loss: 0.3182 - val_loss: 0.2777
Epoch 5/10
15/15 [==============================] - 0s 26ms/step - loss: 0.2526 - val_loss: 0.2372
Epoch 6/10
15/15 [==============================] - 0s 25ms/step - loss: 0.2240 - val_loss: 0.2203
Epoch 7/10
15/15 [==============================] - 0s 26ms/step - loss: 0.2119 - val_loss: 0.2105
Epoch 8/10
15/15 [==============

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape

# Charger le jeu de données MNIST
(x_train, _), (_, _) = mnist.load_data()

# Ici, j'utilise des images aléatoires, mais vous pouvez les remplacer par votre propre dataset
random_images = np.random.rand(10000, 28, 28)
anomalies = random_images.astype('float32') / 255.0
anomalies.shape   # Aplatir les images et les normaliser
X_train = x_train.reshape((len(x_train), 28, 28, 1))
anomalies = anomalies.reshape((len(anomalies), 28, 28, 1))
X_train = X_train / 255.0
anomalies = anomalies / 255.0

# Diviser le jeu de données en ensembles d'entraînement (38 000 images MNIST), de validation (12 000 images MNIST) et de test (10 000 MNIST + 10 000 générées)
X_train, X_val = train_test_split(X_train, test_size=12000, random_state=42)
X_train, X_test = train_test_split(X_train, test_size=10000, random_state=42)

# Concaténer les images de test MNIST avec les anomalies générées
X_test_combined = np.concatenate([X_test, anomalies])

# Créer un autoencodeur convolutionnel
input_img = Input(shape=(28, 28, 1))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(X_train, X_train, epochs=50, batch_size=128, shuffle=True, validation_data=(X_val, X_val))

# Encoder de l'autoencodeur (utilisé comme détecteur d'anomalies)
encoder = Model(input_img, encoded)

# Obtenir les représentations encodées des images d'entraînement
X_train_encoded = encoder.predict(X_train)

# Aplatir les représentations encodées
X_train_encoded_flat = X_train_encoded.reshape((len(X_train_encoded), np.prod(X_train_encoded.shape[1:])))

# Entraîner le modèle One-Class SVM sur les représentations encodées
clf = OneClassSVM(nu=0.1)
clf.fit(X_train_encoded_flat)

# COMPARE la mse du X_train_encoded_flat et le xtrain_flat, 
# si endessous d'un certain seuil c'est une donnee normale si en dessus c'est une anomalie  


# Prédiction sur l'ensemble de test (MNIST + anomalies)
X_test_combined_encoded = encoder.predict(X_test_combined)
X_test_combined_encoded_flat = X_test_combined_encoded.reshape((len(X_test_combined_encoded), np.prod(X_test_combined_encoded.shape[1:])))
y_pred_combined = clf.predict(X_test_combined_encoded_flat)

# Étiquettes réelles de l'ensemble de test (MNIST + anomalies)
y_true_combined = np.concatenate([np.ones(10000), -1 * np.ones(10000)])

# Évaluer la précision sur l'ensemble de test
accuracy_test = accuracy_score(y_true_combined, y_pred_combined)
print(f"Accuracy on combined test set: {accuracy_test}")


Epoch 1/50
297/297 [==============================] - 9s 28ms/step - loss: 0.2525 - val_loss: 0.1653
Epoch 2/50
297/297 [==============================] - 8s 27ms/step - loss: 0.1488 - val_loss: 0.1373
Epoch 3/50
297/297 [==============================] - 8s 27ms/step - loss: 0.1316 - val_loss: 0.1273
Epoch 4/50
297/297 [==============================] - 8s 27ms/step - loss: 0.1240 - val_loss: 0.1217
Epoch 5/50
297/297 [==============================] - 8s 27ms/step - loss: 0.1193 - val_loss: 0.1174
Epoch 6/50
297/297 [==============================] - 8s 27ms/step - loss: 0.1160 - val_loss: 0.1155
Epoch 7/50
297/297 [==============================] - 8s 27ms/step - loss: 0.1135 - val_loss: 0.1123
Ep